In [ ]:
# !pip install -q nltk # for catpro
# !pip install -q -U pip setuptools wheel
# !pip install -q -U 'spacy[apple]' # catpro, change according to device
# !python -m spacy download en_core_web_sm #for catpro
# !pip install -q deplacy==2.0.5
# !pip install -q sentence-transformers==2.2.2
# !pip install -q -U ipywidgets


# Steps

1. Preprocess document (and construct)
- Tokenize into clauses
- Tokenize into words, remove stop words, lemmatize : useful for lexicon count as well

2. Encode construct:
- construct prototype: I want to die
- each token: I wish I didn't wake up tomorrow
- weighted centroid

3. Encode doc


In [ ]:
# # we need to load this here so that preprocessing.embeddings.vectorize runs properly
# import torch
# import flair

In [ ]:
import pandas as pd
import numpy as np
import os
import datetime
import re
import seaborn as sns
import sys
# local scripts
# from preprocessing import stop_words
# from preprocessing import clean
# from preprocessing.tokenizer import spacy_tokenizer
# from preprocessing.lemmatizer import spacy_lemmatizer
# from preprocessing.embeddings import vectorize
# import cts

In [ ]:
# # caution: path[0] is reserved for script path (or '' in REPL)
# sys.path.insert(1, './../catpro')

# from catpro.text.utils import stop_words
# from catpro.text.utils import clean
# from catpro.text.utils.tokenizer import spacy_tokenizer
# from catpro.text.utils.lemmatizer import spacy_lemmatizer

In [ ]:
ts = datetime.datetime.utcnow().strftime('%y-%m-%dT%H-%M-%S')
ts

In [ ]:
input_dir = './../../../data/ctl/input/datasets/'
output_dir = './data/'

task = 'classification'

In [ ]:
import pickle
run_this = False#True saves, False loads
if run_this:
    with open(f'./data/input/ctl/ctl_dfs_features_{task}.pkl', 'wb') as f:
        pickle.dump(dfs, f) 
else:

    with open(f'./data/input/ctl/ctl_dfs_features_{task}.pkl', 'rb') as f:
    	dfs = pickle.load(f)

In [ ]:
# config
construct_representation = 'lexicon' #(lexicon is good for 'prototypes')
doc_representation = 'clauses'
embedding_type = 'sentence'
embedding_model = 'all-MiniLM-L6-v2'
# embedding_package = 'flair'

# Load data

In [ ]:
dfs['test'].keys()

In [ ]:
X_test_df.iloc[:,:2]

In [ ]:
X_test_df = pd.read_csv('./data/input/ctl/X_test_all_with_interaction.csv')
X_test_df = X_test_df.iloc[:,:2]

In [ ]:
metadata_df.columns

In [ ]:
# dvs_for_prompts = {
    
#     'isolate':'lonely',
# }



In [ ]:
# approach_embedding_names = [  # lexicon centroid weighted by distance to construct label
#    # 'w_w_glove', 
#     # 'w_c_psychbert',
#     # 'w_w_minilm', 
#     'w_c_minilm',           
#     # 'wl_w_minilm',
#     # 'wl_c_minilm'
# ]
    

# embedding_name_type = {
# #    model_name:embedding_type 
#     # 'glove': 'word',
#     'all-MiniLM-L6-v2': 'sentence',
#     # 'all-MiniLM-L6-v2': 'document',
#     # 'mnaylor/psychbert-cased': 'document',# need to fix  
# }


    

In [ ]:
# len(docs)

# Preprocessing

1. spell checker: automatically may create a lot of errors. It works by prividing closest orthographic neighbor to words not in a dictionary pyspellchecker is an option.
2. Remove authentification words: TALK, CONNECT, FEEL, BREATHE, HOPELINE, (can be lower case?)


### TODO: remove words with negation 1-3 words prior

In [ ]:
%%time

# Fast: 1 sec every 10 000 messages

run_this = False

if run_this:
    docs =metadata_df['message'].values
    # docs = [re.sub("[\(\[].*?[\)\]]", "", n) for n in docs] #replace text within parentheses/brackets and parentheses/brackets
    # docs = [n.replace('//', '').replace(' .', '.').replace(' ,', ',') for n in docs] 
    # docs = [n.replace('ampx200b', '').replace('\n','').replace('\xa0', '') for n in docs]
    docs_clean = [str(n) if str(n)!='nan' else '' for n in docs]
    docs_clean = [n.replace('!.', '!').replace('?.', '?').replace('....', '...').replace('...', '... ') for n in docs_clean]
    docs_clean = [clean.remove_multiple_spaces(doc) for doc in docs_clean]
    metadata_df['docs_clean'] = docs_clean
    print('made a difference in', metadata_df[metadata_df['message'] != metadata_df['docs_clean']].shape[0]/metadata_df.shape[0], 'samples')

In [ ]:
# more_stop_words = ['ca', 'nt','like', "'", "´", "n’t"]

### tokenize documents into words (remove stop words and lemmatize)

In [ ]:
%%time

run_this = False 

if run_this:
    # words: tokenize by words, remove stop words and lemmatize for word-word similarity
    docs_clean_w_w = stop_words.remove(list(docs_clean), language = 'en', sws = 'nltk', remove_punct=True, extend_stopwords=more_stop_words)
    docs_clean_w_w = [clean.remove_multiple_spaces(doc) for doc in docs_clean_w_w]
    docs_clean_w_w = spacy_lemmatizer(docs_clean_w_w, language ='en') #this takes 22s for 5200 docs
    df['docs_clean_w_w'] = docs_clean_w_w
    


### Tokenize into clauses

In [ ]:
%%time 
# 15m samples for 38k CTL convos



run_this = False


if run_this:
    # docs_clean_clauses = [clean.remove_multiple_spaces(doc) for doc in docs_clean]
    docs_clean_clauses = spacy_tokenizer(docs_clean, 
                                     language = 'en', model='en_core_web_sm', 
                                     token = 'clause', # clause tokenization
                                     lowercase=False, 
                                     display_tree = False, 
                                     remove_punct=True, 
                                     clause_remove_conj = True)
    
    
    metadata_df['docs_clean_clauses'] = docs_clean_clauses
    metadata_df.to_csv(input_dir+f'train10_train_concurrent_metadata_messages_preprocessed_{ts}.csv')
else:
    metadata_df = pd.read_csv(input_dir+f'train10_train_concurrent_metadata_messages_preprocessed_23-07-20T02-00-58.csv')
    

In [ ]:
metadata_df[['message','docs_clean_clauses']].iloc[::10]

In [ ]:
# metadata_df[['message','docs_clean_clauses','docs_clean_w_w']].iloc[::10]


# Encode embeddings and compute similarity

### Construct (Lexicon)

In [ ]:
# # load risk factor and encode

# input_dir_lexicons = './data/input/lexicons/'
# # lexicon = pd.read_csv(input_dir_catpro+'lexicons/suicidal_thoughts_and_behaviors/suicide_risk_lexicon_thesauri_questionnaires_23-03-16T19-35-06.csv', index_col = 0)
# import json

# with open(input_dir_lexicons+'suicide_risk_lexicon_thesauri_questionnaires_23-03-16T19-35-06.json') as f:
#     lexicon = json.load(f)


In [ ]:
# lexicon.keys()

In [ ]:
lexicon_df = pd.read_csv(input_dir_lexicons+'OsirisRankinFirstPassForDanLowMarch_3_ 2023_daniel_added_prototypes.csv', index_col = 0, encoding = 'latin1')

print(len(lexicon_df.columns.tolist()))
print(lexicon_df.columns.tolist()[:10])

constructs_to_measure = ['self-injury',
 'active_si',
 'passive_si',
 'bully',
 'abuse_physical',
 'abuse_sexual',
 'relationships',
 'grief_bereavement',
 'loneliness_isolated',
 'anxiety',
 'depressed_mood',
 'gender_sexual_identity',
 'eating_disorder',
 'substance_use']



# values = ['']*len(constructs_to_measure)
# ctl_tags = dict(zip(constructs_to_measure, values))
ctl_tags_d = {'self-injury': 'self_harm',
 'active_si': 'suicide',
 'passive_si': 'suicide',
 'bully': 'bully',
 'abuse_physical': 'abuse_physical',
 'abuse_sexual': 'abuse_sexual',
 'relationships': 'relationship',
 'grief_bereavement': 'bereavement',
 'loneliness_isolated': 'isolated',
 'anxiety': 'anxiety_stress',
 'depressed_mood': 'depressed',
 'gender_sexual_identity': 'gender_sexual_identity',
 'eating_disorder': 'eating_body_image',
 'substance_use': 'substance', 
  'alcohol_use': 'substance', 
             }

ctl_tags = np.unique(list(ctl_tags_d.values())).tolist()

In [ ]:
lexicon_prototypes = {}

# first create a list for each, so you can append 
for lexicon_construct, ctl_construct in ctl_tags_d.items():
    lexicon_prototypes[ctl_construct] = []



for lexicon_construct, ctl_construct in ctl_tags_d.items():
    print(lexicon_construct, ctl_construct)
    lexicon_df_construct_prototypes = lexicon_df[lexicon_df[lexicon_construct+'_remove'].isin(['prototype', 'seed_token'])][lexicon_construct].tolist()  
    lexicon_prototypes[ctl_construct].extend(lexicon_df_construct_prototypes)
    
lexicon_prototypes
    

# Remove tokens decided by osiris

In [ ]:
# suicide_list_temp = ['I collected pills',
#  'I gave my things away',
#  'I wrote a suicide note',
#  'attempt suicide',
#  'auto exhaust',
#  'commit suicide',
#  'drive my car into a tree',
#  'drown',
#  'firearm',
#  'giving away my possessions',
#  'gun',
#  'hang mysef',
#  'immolation',
#  'jump in front of a train',
#  'jump into traffic',
#  'jump of a bridge',
#  'jump of a building',
#  'jump of my balcony',
#  'jump off a ledge',
#  'kill myself',
#  'make a will',
#  'overdose',
#  'pills',
#  'poison',
#  'rope',
#  'rope around my neck',
#  'set myself on fire',
#  'suffocation',
#  'suicide',
#  'suicide attempt',
#  'suicide note',
#  'take all my pills',
#  'thought about buying a gun',
#  'thought about buying a rope',
#  'thought about buying pills',
#  'thought about hanging my self',
#  'thought about jumping',
#  'weapons',
#  'write a will',
# 'better if I was not alive',
#  'go to sleep and never wake up',
#  'hate my life',
#  'my existence has no direction',
#  'my existence has no meaning',
#  "my life doesn't seem worthwhile",
#  'my life has no clear aims',
#  'my life has no clear goals',
#  'my life has no sense of direction',
#  'my life has no sense of meaning',
#  'my life has no sense of purpose',
#  'my life is going nowhere',
#  'my personal existence has no purpose',
#  'no one cares if I live or die',
#  'nothing to look forward to',
#  'there is no meaning to my life',
#  'there is no purpose in what I am doing',
#  "there's nothing I want to achieve",
#  'want to die',
#  'wish I had never been born',
#  'wish I was never born',
#  'wish I were dead']
                     

In [ ]:
# lexicon_remove = pd.read_csv('./../data/lexicons/suicidal_thoughts_and_behaviors/suicide_risk_lexicon_thesauri_questionnaires_23-03-16T19-35-06_osiris_v01.csv', index_col = 0,encoding='cp1252')
# # suicide_list = lexicon_remove['active_si'].tolist()+lexicon_remove['passive_si'].tolist()
# nans= ['nan']*(lexicon_remove.shape[1]+6)
# lexicon_remove['suicide'] = suicide_list_temp+nans
# lexicon_remove['suicide']

In [ ]:


# lexicon_remove_d = {}                            
# for l in lexicon.keys():
#     l=     ctl_tags_d_inv.get(l)
#     if l=='suicide':
#         lexicon_remove_d[l] = lexicon_remove_i
#         continue

#     lexicon_remove_i = lexicon_remove[[l,l+'_remove']].replace('seed_token', 0).replace('construct label', 1)
#     lexicon_remove_i['self-injury_remove'] = lexicon_remove_i['self-injury_remove'].astype(float)
    
    
#     lexicon_remove_i = lexicon_remove_i[lexicon_remove_i[l+'_remove']>=1][l].values
#     lexicon_remove_i = [n for n in lexicon_remove_i if str(n)!= 'nan' ]
#     print(lexicon_remove_i)    
#     # to_remove_i = []
#     lexicon_remove_d[l] = lexicon_remove_i
    
                             
                             
                             

# define lexicons (old)

In [ ]:
# # merge active and passive
# suicide_tokens = lexicon['active_si']+lexicon['passive_si']
# suicide_tokens = [n for n in suicide_tokens if str(n)!='nan']
# lexicon['suicide'] = suicide_tokens

# # rename lexicon constructs
# for k_old, k_new in ctl_tags_d.items():
#     lexicon[k_new] = lexicon.pop(k_old)

# # keep only ctl constructs
# lexicon_constructs_not_in_ctl = set(lexicon.keys()) - set(ctl_tags_d.values())
# lexicon_constructs_not_in_ctl
# for construct in lexicon_constructs_not_in_ctl:
#     del lexicon[construct]

# # for construct-word to doc analyses, have a single construct 
# # lexicon_prototypes = dict(zip(lexicon.keys(), ['']*len(lexicon.keys())))


# # use prototypes defined in suicide_risk_lexicon_thesauri_questionnaires_23-03-16T19-35-06_osiris_v01.csv 

# lexicon_prototypes = {'suicide': ['suicide', 'kill myself',
#  'self_harm': 'I cut myself, self harm',
#  'bully': "bullied",
#  'abuse_physical': 'physical abuse, violence',
#  'abuse_sexual': 'sexual abuse, rape',
#  'relationship': 'relationship problems',
#  'bereavement': "grieving, mourning",
#  'isolated': 'lonely',
#  'anxiety_stress': 'anxious',
#  'depressed': 'depressed',
#  'gender_sexual_identity': 'gender, queer, gay, sexual orientation',
#  'eating_body_image': 'eating disorder, anorexia, bulimia',
#  'substance': 'drugs, addiction, alcohol, drinking problem'}

word_prototypes = {'suicide': 'suicide',
 'self_harm': 'I cut myself',
 'bully': "bullied",
 'abuse_physical': 'physical abuse',
 'abuse_sexual': 'sexual abuse and rape',
 'relationship': 'relationship',
 'bereavement': "grieving and mourning",
 'isolated': 'lonely',
 'anxiety_stress': 'anxious',
 'depressed': 'depressed',
 'gender_sexual_identity': 'gender and sexual orientation',
 'eating_body_image': 'eating disorder',
 'substance': 'drugs'}



# Encode lexicon

you want to encode each token once, because can appear in multiple lexicons

In [ ]:
embeddings_dir = './data/input/embeddings/'

In [ ]:
# prior_encoded_embeddings = pd.read_csv(input_dir_catpro+'lexicons/suicidal_thoughts_and_behaviors/tokens_embeddings_22-12-02T17-43-57.csv', index_col = 0)
# prior_encoded_embeddings

In [ ]:
# Test run
# embedding_type = 'sentence' 
# docs = ['suicide', 'anxiety', 'sexual abuse and rape']
# result = vectorize(docs, list_of_lists=False, embedding_type = embedding_type, model_name = model_name) # 10 s for list of tokens for 5200 docs

In [ ]:
%%time 



run_this = True




tokens_all = []
for construct, tokens in lexicon_prototypes.items():
    tokens_all.extend(tokens)
model_name = 'all-MiniLM-L6-v2'
embedding_type = 'sentence'
list_of_lists = False

def 
    if run_or_load == 'run':
        embeddings_lexicon_tokens = vectorize(tokens_all, list_of_lists=list_of_lists, embedding_type = embedding_type, model_name = model_name) # 10 s for list of tokens for 5200 docs
        embeddings_lexicon_tokens_d = dict(zip(tokens_all, embeddings_lexicon_tokens))
    
    
        with open(embeddings_dir+f'embeddings_prototypes_{model_name}.pickle', 'wb') as handle:
            pickle.dump(embeddings_lexicon_tokens_d, handle, protocol=pickle.HIGHEST_PROTOCOL)
            
    else:        
        with open(embeddings_dir+f'embeddings_prototypes_{model_name}.pickle', 'rb') as handle:
            embeddings_lexicon_tokens_d = pickle.load(handle)
        # with open(embeddings_dir+f'embeddings_{model_name}.pickle') as f:
        #     embeddings_lexicon_tokens_d = json.load(f)

In [ ]:
# %%time 

# run_this = True

# embeddings_lexicon_tokens_d_protoypes = {}

# if run_this:

#     embeddings_lexicon_prototypes = vectorize(list(lexicon_prototypes.values()), list_of_lists=False, embedding_type = embedding_type, model_name = model_name) # 10 s for list of tokens for 5200 docs

#     for k,v in zip(list(lexicon_prototypes.values()), embeddings_lexicon_prototypes):
#         embeddings_lexicon_tokens_d_protoypes[k]=v
#     # with open(embeddings_dir+f'embeddings_{model_name}.pickle', 'wb') as handle:
#     #     pickle.dump(embeddings_lexicon_tokens_d, handle, protocol=pickle.HIGHEST_PROTOCOL)




# Encode docs

In [ ]:
ctl_tags13 = ['self_harm',
 'suicide',
 'bully',
 'abuse_physical',
 'abuse_sexual',
 'relationship',
 'bereavement',
 'isolated',
 'anxiety_stress',
 'depressed',
 'gender_sexual_identity',
 'eating_body_image',
 'substance']

# identify test sets

run_this = False

datasets_dir = './../../../data/ctl/input/datasets/train10/'

if run_this:
    
    test_set_ids = []
    for dv in ctl_tags13:
        df_i = create_binary_dataset(metadata_df, dv = dv, n_per_dv = 2500, random_state=123)
        train_df,test_df = train_test_split(df_i, test_size=0.2,random_state=123)
        train_df.to_csv(datasets_dir+f'train10_{dv}_train.csv')
        test_df.to_csv(datasets_dir+f'train10_{dv}_test.csv')
        test_set_ids.extend(test_df['conversation_id'].tolist())
    
    test_set_ids = np.unique(test_set_ids)
else:
    test_set_ids = []
    for dv in ctl_tags13:
        train_df = pd.read_csv(datasets_dir+f'train10_{dv}_train.csv', index_col = 0)
        test_df = pd.read_csv(datasets_dir+f'train10_{dv}_test.csv', index_col = 0)
        test_set_ids.extend(test_df['conversation_id'].tolist())
    
    test_set_ids = np.unique(test_set_ids)
    print(test_set_ids.shape)    
    


In [ ]:
metadata_df.columns

In [ ]:
len(test_set_ids)

In [ ]:
#obtained docs tokenized into clauses:
docs_clean_clauses_with_empty_strings = metadata_df[metadata_df['conversation_id'].isin(test_set_ids)]['docs_clean_clauses'].values

docs_clean_clauses_convo_ids = metadata_df[metadata_df['conversation_id'].isin(test_set_ids)]['conversation_id'].values
# IMPORTANT: eval so str becomes list of lists
docs_clean_clauses_with_empty_strings = [eval(n) for n in docs_clean_clauses_with_empty_strings]

# some clauses are empty strings '' . Could fix tokenizer
docs_clean_clauses = []
for doc in docs_clean_clauses_with_empty_strings:
    clean_doc = [ x for x in doc if x != '']
    docs_clean_clauses.append(clean_doc)
    
    





print(len(docs_clean_clauses))

In [ ]:
%%time 
# It's abuout 1m per 100 docs
# 12 m for 1300 docs, each one tokenized
# 100m 11000 docs, each one tokenized


run_this = False



if run_this:
    pickle_name = 'clause_tokens_test_set'
    model_name = 'all-MiniLM-L6-v2'
    embedding_type = 'sentence'
    list_of_lists = True
    verbose = True
    step = 1000

    for i in range(0,len(docs_clean_clauses), step):
        i_str = str(i).zfill(5)
        print(f'vectorizing {i}-{i+step}')
        docs_clean_clauses_i = docs_clean_clauses[i:i+step]
        docs_clean_clauses_convo_ids_i = docs_clean_clauses_convo_ids[i:i+step]
        embeddings_tokens_docs = vectorize(docs_clean_clauses_i, list_of_lists=list_of_lists, embedding_type = embedding_type, model_name = model_name) # 10 s for list of tokens for 5200 docs    
        embeddings_tokens_docs_d = dict(zip(list(docs_clean_clauses_convo_ids_i), embeddings_tokens_docs))
        with open(embeddings_dir+'train10_test/'+f'embeddings_docs_{model_name}_train10_train_concurrent_metadata_messages_{pickle_name}_{ts}_part-{i_str}.pickle', 'wb') as handle:
            pickle.dump(embeddings_tokens_docs_d, handle, protocol=pickle.HIGHEST_PROTOCOL)

    # save into smaller pickles. Then load and add to single pickle
    embeddings_tokens_docs_d = {}
    
    keys_all = []
    
    for i in range(0,len(docs_clean_clauses), step):
        i_str = str(i).zfill(5)
        print(i_str)
        docs_clean_clauses_convo_ids_i = docs_clean_clauses_convo_ids[i:i+step]
        
    
        
        with open(embeddings_dir+'train10_test/'+f'embeddings_docs_{model_name}_train10_train_concurrent_metadata_messages_{pickle_name}_{ts}_part-{i_str}.pickle', 'rb') as handle:
            d_i = pickle.load(handle)
            # the embeddings keys are range(0,1000). Change to convo IDs
            # for i in range(len(d_i)):
            #     new_key = docs_clean_clauses_convo_ids_i[i]
            #     d_i[new_key] = d_i.pop(i)
            embeddings_tokens_docs_d.update(d_i)
            
            # keys_all.append(list(embeddings_tokens_docs_d.keys()))
            
    
    with open(embeddings_dir+'train10_test/'+f'embeddings_docs_{model_name}_train10_train_concurrent_metadata_messages_{pickle_name}_{ts}_part-all.pickle', 'wb') as handle:
                pickle.dump(embeddings_tokens_docs_d, handle, protocol=pickle.HIGHEST_PROTOCOL)



else:
    with open(embeddings_dir+'train10_test/'+'embeddings_docs_all-MiniLM-L6-v2_train10_train_concurrent_metadata_messages_clause_tokens_test_set_23-07-22T22-28-56_part-all.pickle', 'rb') as handle:
    # with open(embeddings_dir+'train10_test/'+'embeddings_docs_all-MiniLM-L6-v2_train10_train_concurrent_metadata_messages_clause_tokens_test_set_23-07-21T16-33-40_part-all.pickle', 'rb') as handle:
        embeddings_tokens_docs_d = pickle.load(handle)

In [ ]:
len(embeddings_tokens_docs_d.keys())

In [ ]:
# # Test: passed
# # take two random sentences. see if re-extracting matches mapping in d. 

# list(embeddings_tokens_docs_d.keys())[10005]
# # 2 = 6934451.0, 10005 = 1856039.0
# test_convo_ids = [6934451.0,  1856039.0]

# test_docs = metadata_df[metadata_df['conversation_id'].isin(test_convo_ids)]['docs_clean_clauses'].values
# test_docs = [eval(n) for n in test_docs]
# test_docs_embeddings = vectorize(test_docs, list_of_lists=list_of_lists, embedding_type = embedding_type, model_name = model_name) # 10 s for list of tokens for 5200 docs    

# for i, emb in enumerate(test_docs_embeddings):
#     assert emb.tolist() == embeddings_tokens_docs_d.get(test_convo_ids[i]).tolist()

In [ ]:
# %%time 

# # 12 m for 1300 docs, each one tokenized


# run_this = False

# if run_this:
#     model_name = 'all-MiniLM-L6-v2'
#     embedding_type = 'sentence'
#     list_of_lists = True
#     verbose = True

#     embeddings_tokens_docs = vectorize(docs_clean_clauses, list_of_lists=list_of_lists, embedding_type = embedding_type, model_name = model_name) # 10 s for list of tokens for 5200 docs    
#     embeddings_tokens_docs_d = dict(zip(list(range(len(docs_clean_clauses))), embeddings_tokens_docs))
#     with open(embeddings_dir+f'embeddings_docs_{model_name}_train10_train_concurrent_metadata_messages_{ts}.pickle', 'wb') as handle:
#         pickle.dump(embeddings_tokens_docs_d, handle, protocol=pickle.HIGHEST_PROTOCOL)
# else:
#     with open(embeddings_dir+'embeddings_docs_all-MiniLM-L6-v2_train10_train_concurrent_metadata_100perconstruct_with_messages.pickle', 'rb') as handle:
#         embeddings_tokens_docs_d = pickle.load(handle)

In [ ]:
# constructs = lexicon.keys()
# construct_prototype_d = lexicon_prototypes
# embeddings_construct_d = embeddings_lexicon_tokens_d
# docs = docs_clean_clauses
# embeddings_docs_d = embeddings_tokens_docs_d
# method = 'word_clause'

### TODO Encode tokes from words 

In [ ]:
# %%time 



# feature_vectors_d = {}

# for method in ['word_word']:
    
#     if method == 'word_word':
#         docs = df['docs_clean_w_w'].values
#     print('=========================')
#     print(method)
#     feature_vectors_all = cts.measure(
#         constructs = lexicon.keys(),
#         construct_prototype_d = lexicon_prototypes,
#         embeddings_construct_d = embeddings_lexicon_tokens_d,
#         docs = docs,
#         embeddings_docs_d = ######TODO,
#         method = method,
#                                                    )
#     feature_vectors_d[method]=feature_vectors_all


### todo: wlexicon for weightedLexicon

### word_ methods for prototype and lexicon_ 

### Check encoding is correct

In [ ]:

# # construct 
# lexicon_prototypes

# construct embeddings
# embeddings_lexicon_tokens_d

# # docs (test set sentences)
# docs_clean_clauses

# # doc embeddings (test set sentence embeddings)
# len(embeddings_tokens_docs_d.keys())




In [ ]:
# obtain one of the construct tokens and their embeddings

tokens = lexicon_prototypes.get('self_harm') 
print(tokens)
token_embeddings = np.array([embeddings_lexicon_tokens_d.get(token) for token in tokens]) # obtain one of the construct tokens
print(token_embeddings.shape)
token_embeddings2 = vectorize(tokens)

assert token_embeddings == token_embeddings2



In [ ]:
# obtain one of the doc tokens and their embeddings
df_example = metadata_df[metadata_df['docs_clean_clauses'].str.contains("Feeling very suicidal and about to take pills just need someone|i feel alone and")]
# display(df_example)
docs_i = [eval(n) for n in df_example['docs_clean_clauses'].values]
docs_i_clean = []
for doc in docs_i:
    clean_doc = [ x for x in doc if x != '']
    docs_i_clean.append(clean_doc)
    
docs_i_embeddings = vectorize(docs_i_clean, list_of_lists=True)
docs_i_embeddings_d = dict(zip(df_example['conversation_id'].values, docs_i_embeddings))

In [ ]:
lexicon_protoypes

In [ ]:
from importlib import reload
reload(cts)

method = 'lexicon_clause'
feature_vectors_i, cosine_scores_docs_i = cts.measure(
            construct_tokens_d = construct_tokens_d,
    		construct_embeddings_d = construct_embeddings_d,
    		docs = docs_i,
    		docs_embeddings_d = docs_i_embeddings_d,
    		method = method, #todo: change to token, tokens, weighted_tokens
    		summary_stat = ['max']
        )

feature_vectors_i

In [ ]:
# Test: passed, 
construct = 'bully'
feature_vectors_all_copy = feature_vectors_all.copy()
feature_vectors_i_d = dict(zip(feature_vectors_i['doc_id'].values, feature_vectors_i[construct+'_max'].values))
# feature_vectors_all_copy[construct+'_max2'] = feature_vectors_all_copy['doc_id'].map(feature_vectors_i_d)
# feature_vectors_all_copy[~feature_vectors_all_copy[construct+'_max2'].isna()][[construct+'_max', construct+'_max2']]

In [ ]:

ctl_tags13

In [ ]:
doc_id

In [ ]:
feature_vectors_all[feature_vectors_all['doc_id']==doc_id]

In [ ]:


# for doc_id, score in feature_vectors_i_d.items():
#     print('============='*3)
#     print('doc_id', doc_id)
#     metadata_df_i = metadata_df[metadata_df['conversation_id']==doc_id]
#     for tag in ctl_tags13:
#         if metadata_df_i[tag].values[0]==1:
#             score = feature_vectors_i[feature_vectors_i['doc_id']==doc_id][tag+'_max'].round(2).values[0]
            
#             print(tag, 'true', 1, '\tcts predicted', score)
#     display(np.array(eval(metadata_df_i['docs_clean_clauses'].values[0])))


In [ ]:
docs_clean_clauses_i = metadata_df['conversation_id']==1909812.0

# Construct text similarity (once construct and doc are encoded)

In [ ]:
test_df[test_df['conversation_id']==3137630.0]

In [ ]:
# any non strings in tokens?
[type(n) for n in tokens_all if type(n)!=str]

In [ ]:
# Any empty tokens?
[n[:2] for n in np.array(list(embeddings_lexicon_tokens_d.values()))]

### compute with word prototypes (see below for proper code, then turn that into a function)

In [ ]:
from importlib import reload
reload(cts)
# toy

# doc_i = eval(metadata_df[metadata_df['message'].str.contains('skinny and beautiful')]['docs_clean_clauses'].values[0])[:5]
docs_clean_clauses_i = [["I'm scared that I might develop an eating disorder like anorexia", 'Yea for as long as I could remember I felt fat', 'my family said that I never looked pretty because I weighed so much', 'And now in college I walk around', 'see girls who are extremely skinny and beautiful']]
docs_clean_clauses_i_embeddings = vectorize(docs_clean_clauses_i, list_of_lists=True)

embeddings_tokens_docs_d_i = {12345678:docs_clean_clauses_i_embeddings[0]}

print(docs_clean_clauses_i)

feature_vectors_all,cosine_scores_docs_all = cts.measure(
        construct_tokens_d = construct_tokens_d,
        construct_embeddings_d = construct_embeddings_d,
        docs = docs_clean_clauses_i,
        docs_embeddings_d = embeddings_tokens_docs_d_i,
        method = method, #todo: change to token, tokens, weighted_tokens
        summary_stat = ['max'],
    return_cosine_similarity=True
    
    )


feature_vectors_all
print(cosine_scores_docs_all)
doc_id = 12345678
construct = 'eating_body_image'



In [ ]:
feature_vectors_all

### compute with lexicon prototypes


In [ ]:
tokens_all = []
for construct, tokens in lexicon_prototypes.items():
    tokens_all.extend(tokens)
embeddings_lexicon_tokens = vectorize(tokens_all, list_of_lists=list_of_lists, embedding_type = embedding_type, model_name = model_name) # 10 s for list of tokens for 5200 docs
embeddings_lexicon_tokens_d = dict(zip(tokens_all, embeddings_lexicon_tokens))
construct_tokens_d = lexicon_prototypes.copy()
construct_embeddings_d = embeddings_lexicon_tokens_d.copy()

from importlib import reload
reload(cts)
# toy

# doc_i = eval(metadata_df[metadata_df['message'].str.contains('skinny and beautiful')]['docs_clean_clauses'].values[0])[:5]
docs_clean_clauses_i = [["I'm scared that I might develop an eating disorder like anorexia", 'Yea for as long as I could remember I felt fat', 'my family said that I never looked pretty because I weighed so much', 'And now in college I walk around', 'see girls who are extremely skinny and beautiful']]
docs_clean_clauses_i_embeddings = vectorize(docs_clean_clauses_i, list_of_lists=True)

embeddings_tokens_docs_d_i = {12345678:docs_clean_clauses_i_embeddings[0]}

print(docs_clean_clauses_i)

feature_vectors_all,cosine_scores_docs_all = cts.measure(
        construct_tokens_d = construct_tokens_d,
        construct_embeddings_d = construct_embeddings_d,
        docs = docs_clean_clauses_i,
        docs_embeddings_d = embeddings_tokens_docs_d_i,
        method = 'lexicon_clause', #todo: change to token, tokens, weighted_tokens
        summary_stat = ['max'],
    return_cosine_similarity=True
    
    )


feature_vectors_all
print(cosine_scores_docs_all)
doc_id = 12345678
construct = 'eating_body_image'




In [ ]:
[print(n, '\n') for n in np.round(cosine_scores_docs_all.get(str(doc_id)+'_'+construct),2)]

In [ ]:
word_prototypes

In [ ]:
[print(n, '\n') for n in docs_clean_clauses_i[0]]

In [ ]:
len(tokens_all)/13

In [ ]:
# feature_vectors_all.to_csv(f'./data/input/datasets/cts_prototypes-clauses_prototypes_clause_{ts}.csv')

In [ ]:
word_prototypes.items()

In [ ]:
feature_vectors_all[1]

In [ ]:
feature_vectors_all[0].to_csv(f'./data/input/datasets/cts_testsets_{method}_clause_{ts}.csv')
with open(f'./data/input/datasets/cts_testsets_{method}_clause_{ts}.pkl', 'wb') as f:
    pickle.dump(feature_vectors_d, f)

In [ ]:
%%time 
# if construct==1 word, then 10k takes about 1 min
# if construct==about 10 words on average, then 10k takes about 17 min
from importlib import reload
reload(cts)

method = 'word_clause' #['word_clause', 'lexicon_clause']
run_this = False


model_name = 'all-MiniLM-L6-v2'
embedding_type = 'sentence'
list_of_lists = False

if run_this:

    

    print('=========================')
    print(method)
    if method == 'lexicon_clause':
        # encode construct
        tokens_all = []
        for construct, tokens in lexicon_prototypes.items():
            tokens_all.extend(tokens)
        embeddings_lexicon_tokens = vectorize(tokens_all, list_of_lists=list_of_lists, embedding_type = embedding_type, model_name = model_name) # 10 s for list of tokens for 5200 docs
        embeddings_lexicon_tokens_d = dict(zip(tokens_all, embeddings_lexicon_tokens))
        
        # define dicttionaries
        construct_tokens_d = lexicon_prototypes.copy()
        construct_embeddings_d = embeddings_lexicon_tokens_d.copy()

    elif method == 'word_clause':
        # encode construct
        tokens_all = word_prototypes.values()
        embeddings_word_tokens = vectorize(tokens_all, list_of_lists=list_of_lists, embedding_type = embedding_type, model_name = model_name) # 10 s for list of tokens for 5200 docs
        embeddings_word_tokens_d = dict(zip(tokens_all, embeddings_word_tokens))
        
        # define dicttionaries
        construct_tokens_d = word_prototypes.copy()
        construct_embeddings_d = embeddings_word_tokens_d.copy()
        
    feature_vectors_all = cts.measure(
        construct_tokens_d = construct_tokens_d,
        construct_embeddings_d = construct_embeddings_d,
        docs = docs_clean_clauses,
        docs_embeddings_d = embeddings_tokens_docs_d,
        method = method, #todo: change to token, tokens, weighted_tokens
        summary_stat = ['max']
    )
    feature_vectors_all.to_csv(f'./data/input/datasets/cts_testsets_{method}_clause_{ts}.csv')
    with open(f'./data/input/datasets/cts_testsets_{method}_clause_{ts}.pkl', 'wb') as f:
        pickle.dump(feature_vectors_d, f)


else:
    if method == 'word_clause':
        feature_vectors_all = pd.read_csv(f'./data/input/datasets/cts_testsets_word_clause_clause_23-07-22T22-28-56.csv')
    elif method == 'lexicon_clause':
        feature_vectors_all = pd.read_csv(f'./data/input/datasets/cts_testsets_lexicon_clause_23-07-22T22-28-56.csv')
    
    # # with open(f'./data/input/datasets/cts_testsets_prototypes_clause_23-07-22T16-39-15.pkl', 'rb') as f:
    # with open(f'./data/input/datasets/cts_testsets_{method}_clause_23-07-22T16-39-15.pkl', 'rb') as f:
    #     feature_vectors_d = pickle.load(f)

    


In [ ]:
feature_vectors_all = feature_vectors_all.sort_values('doc_id')
feature_vectors_all2 == feature_vectors_all

In [ ]:


idx = 3137630.0

print(metadata_df[metadata_df['conversation_id']==idx]['message'].values)

feature_vectors_all[feature_vectors_all['doc_id']==idx]

In [ ]:
# %%time 



# feature_vectors_d = {}

# for method in ['lexicon_clause']:#['word_clause', 'lexicon_clause']:
#     print('=========================')
#     print(method)
#     feature_vectors_all = cts.measure(
#         constructs = lexicon.keys(),
#         construct_prototype_d = lexicon_prototypes,
#         embeddings_construct_d = embeddings_lexicon_tokens_d,
#         docs = docs_clean_clauses,
#         embeddings_docs_d = embeddings_tokens_docs_d,
#         method = method,
#                                                    )
#     feature_vectors_d[method]=feature_vectors_all


In [ ]:
# !pip install -q dcor

In [ ]:
import other_metrics

In [ ]:
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, roc_auc_score

In [ ]:
from sklearn import metrics
from scipy.special import softmax
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc

# calculate the fpr and tpr for all thresholds of the classification
# probs = model.predict_proba(X_test)
# preds = probs[:,1]

# Softmax. not really making sense.
# https://discuss.pytorch.org/t/logits-vs-log-softmax/95979


figsize = (4,4)



In [ ]:
# feature_vectors_all = feature_vectors_d.get(method) # DF feature vector for each doc, the max score with each construct
# feature_vectors_all_d = dict(zip(feature_vectors_all['doc_id'], feature_vectors_all[construct+'_max']))
# print(len(feature_vectors_all_d.keys()))

In [ ]:
# construct = 'isolated'
# test_df = pd.read_csv(datasets_dir+f'train10_{construct}_test.csv', index_col = 0)
# test_df[construct+'_max'] = test_df['conversation_id'].map(feature_vectors_all_d)
# test_df[['message', construct, construct+'_max']].values[5:12]

In [ ]:
# lexicon_prototypes.get('suicide')

In [ ]:
# docs_toy = ['Feeling very suicidal and about to take pills just need someone to talk to','take all my pills', 'suicide', 'I suppose my husband finally figured out I was trying to leave and threatened to kill himself.', 'kill myself', 'kill']
# embeddings_toy = vectorize(docs_toy)


In [ ]:
# from sklearn.metrics.pairwise import cosine_similarity
# cosine_similarity(embeddings_toy)

In [ ]:
# construct = 'relationship'
# # lexicon_prototypes
# # embeddings_lexicon_tokens_d
# construct_i_embeddings = []
# for token in lexicon_prototypes.get(construct):
#     construct_i_embeddings.append(embeddings_lexicon_tokens_d.get(token))



In [ ]:
# docs_i = metadata_df[metadata_df['conversation_id'].isin([7263937.0, 3870339.0])]['docs_clean_clauses'].values
# docs_i = [eval(n) for n in docs_i]
# docs_i_embeddings = vectorize(docs_i, list_of_lists=True)


In [ ]:
# construct_i_embeddings = np.stack(construct_i_embeddings, axis=0).astype(float)
# for docs_i_embeddings_i in docs_i_embeddings:
#     docs_i_embeddings_i = np.stack(docs_i_embeddings_i, axis=0).astype(float)
#     cosine_scores_docs_i = cosine_similarity(construct_i_embeddings, docs_i_embeddings_i)

In [ ]:
# print(construct_i_embeddings.shape)
# print(docs_i_embeddings_i.shape)
# cosine_scores_docs_i.shape

In [ ]:
# np.max(cosine_scores_docs_i, axis=1)

In [ ]:
# np.max(cosine_scores_docs_i)


In [ ]:
# feature_vectors_all_d = dict(zip(feature_vectors_all['doc_id'], feature_vectors_all[construct+'_max']))

# docs_toy = ['Feeling very suicidal and about to take pills just need someone to talk to','take all my pills', 'suicide', 'I suppose my husband finally figured out I was trying to leave and threatened to kill himself.', 'kill myself', 'kill']
# embeddings_toy = vectorize(docs_toy)



In [ ]:
ctl_tags13

In [ ]:
# metric_names = ['Precision','Recall', 'Specificity', 'F1','Avg. Precision', 'ROC AUC']
# results_05 = pd.DataFrame(results_all_05, index = metric_names).T.round(3)
# print(results_05.mean())
# results_05.loc['Mean', results_05.columns] = results_05.mean().round(3)
# display(results_05)

# results_045 = pd.DataFrame(results_all_045, index =metric_names).T.round(3)
# results_045.loc['Mean', results_045.columns] = results_045.mean().round(3)
# display(results_045)

# results_04 = pd.DataFrame(results_all_04, index =metric_names).T.round(3)
# results_04.loc['Mean', results_04.columns] = results_04.mean().round(3)
# display(results_04)

In [ ]:
def metrics_to_df(y_true, y_pred,y_proba=None, model_name='Model X', add_stats = True, round_to_decimal = 3):
	# todo: add metric options
	results_d = {}
	precision = metrics.precision_score(y_true,y_pred,average = 'binary')
	recall = metrics.recall_score(y_true,y_pred,average = 'binary')
	specificity = metrics.recall_score(y_true, y_pred, pos_label=0,average = 'binary')
	f1 = metrics.f1_score(y_true,y_pred,average = 'binary')
	avg_pr = metrics.average_precision_score(y_true, y_proba)
	roc_auc = metrics.roc_auc_score(y_true,y_proba)
	results_d[model_name] = [precision, recall,specificity, f1, avg_pr, roc_auc]
	metric_names = ['Precision','Recall', 'Specificity', 'F1','Avg. Precision', 'ROC AUC']
	results_df = pd.DataFrame(results_d, index = metric_names).T.round(round_to_decimal)
	results_df.mean()
	if add_stats:
		results_df.loc['Mean', results_df.columns] = results_df.mean().round(round_to_decimal)
        results_df.loc['Min', results_df.columns] = results_df.min().round(round_to_decimal)
        results_df.loc['Max', results_df.columns] = results_df.max().round(round_to_decimal)
        
	return results_df

In [ ]:
%%time
from importlib import reload
reload(other_metrics)

import time 

dcor_all = []
    
r_all = []
results_all_05 = {}
results_all_045 = {}
results_all_04 = {}

from sklearn import metrics
plot_roc_pr = False
plot_density = False # hanging here for some reason
results_d = {}

# for method in ['lexicon_clause']:# ['word_clause', lexicon_clause']
print(method)
# feature_vectors_all = feature_vectors_d.get(method) # DF feature vector for each doc, the max score with each construct

# tags = metadata_df[constructs] # ground truth


# max_cols = [n+'_max' for n in constructs]
# feature_vectors_all_max = feature_vectors_all[max_cols]
# assert [n+'_max' for n in constructs] == max_cols  #make sure they're in the right order
# feature_vectors_all_max.columns = constructs        
# dcor
# dcor_i = dcor.distance_correlation(tags,feature_vectors_all_max) 
# dcor_all.append(dcor_i)
# print('dcor', dcor_i)

r_method_i = []
results_d = {}
for construct in ctl_tags13:# ctl_tags13:#['self_harm']: #:
    print('==='*80)
    print(construct)
    # training_samples = 300 # up to 4000
    # train_df = pd.read_csv(datasets_dir+f'train10_{construct}_train.csv', index_col = 0).sample(n=training_samples)
    # y_train = train_df[construct].values
    test_df = pd.read_csv(datasets_dir+f'train10_{construct}_test.csv', index_col = 0)
    feature_vectors_all_d = dict(zip(feature_vectors_all['doc_id'], feature_vectors_all[construct+'_max']))
    test_df[construct+'_max'] = test_df['conversation_id'].map(feature_vectors_all_d)
    # test_df[['conversation_id', construct, construct+'_max']]
    y_true = test_df[construct].values
    y_pred = test_df[construct+'_max'].values

    # y_true_1 = tags[tags[construct]==1] # obtain N samples that where construct == 1, others should == 0 due to preprocessing
    # y_true_0 = tags[tags[construct]==0].sample(n=y_true_1.shape[0]) # match to y_true_1 N in length
    # y_true_1_indexes = y_true_1.index.tolist()
    # y_true_0_indexes = y_true_0.index.tolist()
    # y_true = pd.concat([y_true_1,y_true_0],axis=0)[construct].values # combine y=0 and y=1 and same size
    # y_pred = feature_vectors_all_max[construct][y_true_1_indexes+y_true_0_indexes].values

    

    # dcor_i = dcor.distance_correlation(y_true,y_pred) #0.45
    # print(construct, f'dcor={np.round(dcor_i,2)}', np.round(r,2), np.round(p, 4))

    # df_i = pd.DataFrame(feature_vectors_all_max[construct][y_true_1_indexes+y_true_0_indexes])
   
    

    #here y_pred is max cosine similarity 
    roc_auc = roc_auc_score(y_true,y_pred)
    avg_pr = average_precision_score(y_true, y_pred)
    print('roc auc', np.round(roc_auc,3))
    print('avg_pr', np.round(avg_pr,3))
    if plot_roc_pr:
        metricsother_metrics.plot_roc_auc_curve(y_true, y_pred)
        other_metrics.plot_precision_recall_curve(y_true, y_pred)
    
    
    
    test_df['05-threshold'] = test_df[construct+'_max']>=0.5
    test_df['045-threshold'] = test_df[construct+'_max']>=0.45
    test_df['04-threshold'] = test_df[construct+'_max']>=0.4

    # correlation 
    r, p = pointbiserialr(y_true,y_pred)
    if p < 0.001:
        p = '***'
    elif 0.01 > p >= 0.001:
        p = '**'
    elif 0.05 > p >= 0.01:
        p = '*'
    elif p > 0.05:
        p = ''
        
    

        
    

    # class_report = classification_report(y_true,y_pred)
    # print(construct)
    # print(class_report)

    
    
            
    # Cohens D
    # ===========================
    y_pred_0 = test_df[test_df[construct]==0][construct+'_max'].values
    y_pred_1 = test_df[test_df[construct]==1][construct+'_max'].values
    # cohens_d = (np.mean(y_pred_1) - np.mean(y_pred_0)) / (np.sqrt((np.std(y_pred_1) ** 2 + np.std(y_pred_0) ** 2) / 2))
    cohens_d = other_metrics.cohend(y_pred_1,y_pred_0)
    # rocauc = roc_auc_score(y_true,y_pred)
    # rocauc = str(np.round(roc_auc_score,2))
    title_i = f"{construct}: rho={np.round(r,2)}{p} Cohen's {np.round(cohens_d,2)}"
    print(title_i)
    
    if plot_density: 
        
        # display(df_i)
        sns.kdeplot(data=test_df,x = construct+'_max', hue=construct)
        plt.title(title_i)
        plt.show()
    # r_method_i.append(r)


        
    # r_all_method_i_stats.append(
        
    
    for threshold in ['05-threshold', '04-threshold', '045-threshold']:
        y_pred = test_df[threshold].values
        y_true = test_df[construct].values
        f1 = f1_score(y_true,y_pred,average = 'binary')
        precision = precision_score(y_true,y_pred,average = 'binary')
        recall = recall_score(y_true,y_pred,average = 'binary')
        specificity = metrics.recall_score(y_true, y_pred, pos_label=0,average = 'binary')
        

            

        if '05-' in threshold:
            results_all_05[construct] = [precision, recall,specificity, f1, avg_pr, roc_auc, r, cohens_d]
        elif '04-' in threshold:
            results_all_04[construct] = [precision, recall, specificity, f1, avg_pr,roc_auc, r, cohens_d]
        elif '045-' in threshold:
            results_all_045[construct] = [precision, recall, specificity, f1, avg_pr ,roc_auc, r, cohens_d]

# r_all.append(r_method_i)
# stats = np.round([np.mean(r_all), np.std(r_all),np.min(r_all),np.max(r_all)],2)
# print(f'stats: {stats[0]} ± {stats[1]} ({stats[2]}-{stats[3]})')
# print()

In [ ]:
# metric_names = ['Precision','Recall', 'Specificity','F1',  'ROC AUC']
# results = pd.DataFrame(results_d, index = metric_names).T.round(3)
# results_mean = results.mean().round(3)
# results_min = results.min().round(3)
# results_max = results.max().round(3)


In [ ]:
results_05.columns

In [ ]:
reload(other_metrics)

In [ ]:
from other_metrics import su

In [ ]:
output_dir = './data/output/construct_text_similarity/'

In [ ]:
metric_names = ['Precision','Recall', 'Specificity', 'F1','Avg. Precision', 'ROC AUC', 'point-biserial', "Cohen's D"]


results_05 = pd.DataFrame(results_all_05, index = metric_names).T.round(2)
# results_05.loc['Mean', results_05.columns] = results_05.mean().round(2)
results_05.loc['Mean [min-max]', results_05.columns] = other_metrics.summary_metrics_df(results_05).values[0]
results_05.to_csv(output_dir+f'results_train10_{method}_threshold-050.csv')
display(results_05)

results_045 = pd.DataFrame(results_all_045, index =metric_names).T.round(2)
results_045.loc['Mean [min-max]', results_045.columns] = other_metrics.summary_metrics_df(results_045).values[0]
results_045.to_csv(output_dir+f'results_train10_{method}_threshold-045.csv')
display(results_045)

results_04 = pd.DataFrame(results_all_04, index =metric_names).T.round(2)
results_04.loc['Mean [min-max]', results_04.columns] = other_metrics.summary_metrics_df(results_04).values[0]
results_04.to_csv(output_dir+f'results_train10_{method}_threshold-040.csv')
display(results_04)

In [ ]:
results_04.columns

# correlation with presence of words


In [ ]:
ctl_tags13

In [ ]:
%%time
from importlib import reload
reload(other_metrics)

words_pointbiserial = {}

import time 

dcor_all = []
    
r_all = []
results_all_05 = {}
results_all_045 = {}
results_all_04 = {}

from sklearn import metrics
plot_roc_pr = False
plot_density = False # hanging here for some reason
results_d = {}

# for method in ['lexicon_clause']:# ['word_clause', lexicon_clause']
print(method)
# feature_vectors_all = feature_vectors_d.get(method) # DF feature vector for each doc, the max score with each construct

# tags = metadata_df[constructs] # ground truth


# max_cols = [n+'_max' for n in constructs]
# feature_vectors_all_max = feature_vectors_all[max_cols]
# assert [n+'_max' for n in constructs] == max_cols  #make sure they're in the right order
# feature_vectors_all_max.columns = constructs        
# dcor
# dcor_i = dcor.distance_correlation(tags,feature_vectors_all_max) 
# dcor_all.append(dcor_i)
# print('dcor', dcor_i)

r_method_i = []
results_d = {}
words = ['eating', 'body', 'fat','ugly', 'binge', 'skinny', 'gained', 'anorexia', 'image', 'struggle', 'purging', 'starving','bulimia']

for word_i in words:
    
    for construct in ['eating_body_image']:#ctl_tags13:# ctl_tags13:#['self_harm']: #:
        print('==='*80)
        print(construct)
        # training_samples = 300 # up to 4000
        # train_df = pd.read_csv(datasets_dir+f'train10_{construct}_train.csv', index_col = 0).sample(n=training_samples)
        # y_train = train_df[construct].values
        test_df = pd.read_csv(datasets_dir+f'train10_{construct}_test.csv', index_col = 0)
        feature_vectors_all_d = dict(zip(feature_vectors_all['doc_id'], feature_vectors_all[construct+'_max']))
        test_df[construct+'_max'] = test_df['conversation_id'].map(feature_vectors_all_d)
        # test_df[['conversation_id', construct, construct+'_max']]
        y_true = test_df[construct].values
        y_pred = test_df[construct+'_max'].values
    
        # y_true_1 = tags[tags[construct]==1] # obtain N samples that where construct == 1, others should == 0 due to preprocessing
        # y_true_0 = tags[tags[construct]==0].sample(n=y_true_1.shape[0]) # match to y_true_1 N in length
        # y_true_1_indexes = y_true_1.index.tolist()
        # y_true_0_indexes = y_true_0.index.tolist()
        # y_true = pd.concat([y_true_1,y_true_0],axis=0)[construct].values # combine y=0 and y=1 and same size
        # y_pred = feature_vectors_all_max[construct][y_true_1_indexes+y_true_0_indexes].values
    
        
    
        # dcor_i = dcor.distance_correlation(y_true,y_pred) #0.45
        # print(construct, f'dcor={np.round(dcor_i,2)}', np.round(r,2), np.round(p, 4))
    
        # df_i = pd.DataFrame(feature_vectors_all_max[construct][y_true_1_indexes+y_true_0_indexes])
       
        
    
        #here y_pred is max cosine similarity 
        roc_auc = roc_auc_score(y_true,y_pred)
        avg_pr = average_precision_score(y_true, y_pred)
        print('roc auc', np.round(roc_auc,3))
        print('avg_pr', np.round(avg_pr,3))
        if plot_roc_pr:
            metricsother_metrics.plot_roc_auc_curve(y_true, y_pred)
            other_metrics.plot_precision_recall_curve(y_true, y_pred)
        
    
    
        test_df['presence_of_word_i'] = test_df['message'].str.contains(word_i).values
        from scipy.stats import pointbiserialr
        r,p = pointbiserialr(test_df['presence_of_word_i'], y_pred)
        print(word_i, np.round(r,2)    )
        words_pointbiserial[word_i] = np.round(r,2)
    
pd.DataFrame(words_pointbiserial, index= ['cts']).T.to_csv(output_dir+'point_biserial_top_words.csv')
    
    
        
        
        # test_df['05-threshold'] = test_df[construct+'_max']>=0.5
        # test_df['045-threshold'] = test_df[construct+'_max']>=0.45
        # test_df['04-threshold'] = test_df[construct+'_max']>=0.4
    
        # # correlation 
        # r, p = pointbiserialr(y_true,y_pred)
        # if p < 0.001:
        #     p = '***'
        # elif 0.01 > p >= 0.001:
        #     p = '**'
        # elif 0.05 > p >= 0.01:
        #     p = '*'
        # elif p > 0.05:
        #     p = ''
            
        
    
            
        
    
        # # class_report = classification_report(y_true,y_pred)
        # # print(construct)
        # # print(class_report)
    
        
        
                
        # # Cohens D
        # # ===========================
        # y_pred_0 = test_df[test_df[construct]==0][construct+'_max'].values
        # y_pred_1 = test_df[test_df[construct]==1][construct+'_max'].values
        # # cohens_d = (np.mean(y_pred_1) - np.mean(y_pred_0)) / (np.sqrt((np.std(y_pred_1) ** 2 + np.std(y_pred_0) ** 2) / 2))
        # cohens_d = other_metrics.cohend(y_pred_1,y_pred_0)
        # # rocauc = roc_auc_score(y_true,y_pred)
        # # rocauc = str(np.round(roc_auc_score,2))
        # title_i = f"{construct}: rho={np.round(r,2)}{p} Cohen's {np.round(cohens_d,2)}"
        # print(title_i)
        
        # if plot_density: 
            
        #     # display(df_i)
        #     sns.kdeplot(data=test_df,x = construct+'_max', hue=construct)
        #     plt.title(title_i)
        #     plt.show()
        # # r_method_i.append(r)
    
    
            
        # r_all_method_i_stats.append(
            
        
        # for threshold in ['05-threshold', '04-threshold', '045-threshold']:
        #     y_pred = test_df[threshold].values
        #     y_true = test_df[construct].values
        #     f1 = f1_score(y_true,y_pred,average = 'binary')
        #     precision = precision_score(y_true,y_pred,average = 'binary')
        #     recall = recall_score(y_true,y_pred,average = 'binary')
        #     specificity = metrics.recall_score(y_true, y_pred, pos_label=0,average = 'binary')
            
    
                
    
        #     if '05-' in threshold:
        #         results_all_05[construct] = [precision, recall,specificity, f1, avg_pr, roc_auc, r, cohens_d]
        #     elif '04-' in threshold:
        #         results_all_04[construct] = [precision, recall, specificity, f1, avg_pr,roc_auc, r, cohens_d]
        #     elif '045-' in threshold:
        #         results_all_045[construct] = [precision, recall, specificity, f1, avg_pr ,roc_auc, r, cohens_d]
    
    # r_all.append(r_method_i)
    # stats = np.round([np.mean(r_all), np.std(r_all),np.min(r_all),np.max(r_all)],2)
    # print(f'stats: {stats[0]} ± {stats[1]} ({stats[2]}-{stats[3]})')
    # print()

# OLD

In [ ]:
metric_names

In [ ]:
sns.kdeplot(data=df_i,x = construct, hue='Truth')

In [ ]:
df_i

In [ ]:
# max_cols = [n+'_max' for n in constructs]

# for c in constructs:
#     print(c)
#     feature_vectors_all_max_docs = feature_vectors_all[['doc']+max_cols].sort_values([construct+'_max'])[::-1]['doc'].values[:5]
#     [print('. '.join(eval(n)), '\n') for n in feature_vectors_all_max_docs]
#     # print(feature_vectors_all_max_docs)
    

In [ ]:
# for idx in feature_vectors_all.index:
#     truth = dict(zip(constructs, df[constructs].iloc[idx,:].values))
#     print(truth)
    
#     features = feature_vectors_all.iloc[idx,:].values
#     print('. '.join(eval(features[0])))
#     print(features[1:])
          
#     max_cols = [n+'_max' for n in constructs]
    

In [ ]:
import dcor

In [ ]:
%%time 


In [ ]:
# %%time 

# tags = df[constructs]

# max_cols = [n+'_max' for n in constructs]
# word_clause_max = feature_vectors_all[max_cols]
# assert [n+'_max' for n in constructs] == max_cols 
# word_clause_max.columns = constructs

In [ ]:
word_clause_max

In [ ]:
dcor_lexicon_clause_max = dcor.distance_correlation(tags,lexicon_clause_max) #0.45

In [ ]:
dcor_lexicon_clause_max

In [ ]:
word_clause_max.isna().sum()

In [ ]:
assert tags.columns.tolist() == word_clause_max.columns.tolist()

In [ ]:



for construct in constructs:
    y_true_1 = tags[tags[construct]==1]
    y_true_0 = tags[tags[construct]==0].sample(n=y_true_1.shape[0])
    y_true_1_indexes = y_true_1.index.tolist()
    y_true_0_indexes = y_true_0.index.tolist()
    y_true = pd.concat([y_true_1,y_true_0],axis=0)[construct].values
    # y_true = y_true.sample(n=y_true.shape[0])
    y_pred = word_clause_max[construct][y_true_1_indexes+y_true_0_indexes].values
    r, p = spearmanr(y_true,y_pred)
    print(construct, np.round(r,2), np.round(p, 4))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
sns.kdeplotword_clause_max[construct][y_true_1_indexes].values
word_clause_max[construct][y_true_0_indexes].values

In [ ]:
for construct in constructs:
    y_true = tags[construct].values
    y_pred = word_clause_max[construct]
    r, p = spearmanr(y_true,y_pred)
    print(construct, np.round(r,2), np.round(p, 4))
    

In [ ]:
for construct in constructs:
    y_true = tags[construct].values
    y_pred = word_clause_max[construct]
    r, p = spearmanr(y_true,y_pred)
    print(construct, np.round(r,2), np.round(p, 4))

In [ ]:
df

In [ ]:
# %%time
# # embeddings_tokens_docs ={}

# feature_vectors_all = []

# for i, doc in enumerate(docs_clean_clauses[:10]):
#     if i%100==0:
#         print(i)
#     embeddings_tokens_doc_i = vectorize(doc, list_of_lists=list_of_lists, embedding_type = embedding_type, model_name = model_name) # 10 s for list of tokens for 5200 docs    

In [ ]:
%%time
# embeddings_tokens_docs ={}

feature_vectors_all = []


embeddings_tokens_doc_i = vectorize(docs_clean_clauses[:10], list_of_lists=list_of_lists, embedding_type = embedding_type, model_name = model_name) # 10 s for list of tokens for 5200 docs    

In [ ]:
len(embedding_construct_prototype.shape) == 1

In [ ]:
len(feature_vectors_all)

In [ ]:

    
#     for i in docs_embeddings:
        
    
    
    
    
    
# # def df_similarity_token_category(embeddings_tokens_doc, constructs_d, df, docs_clean, summary_stats = None):
# #     '''
# #     embeddings_tokens_doc
# #     '''
# # for each doc, it creates a value (e.g., mean across tokens --either words or clauses) for each construct    

# feature_vectors_mean = []
# feature_vectors_median = []
# feature_vectors_max = []

# constructs = list(constructs_d.keys())

# for i, doc in enumerate(docs_clean):
#     embeddings_tokens_doc_i = embeddings_tokens_doc[i]
#     df_scores_category_all = pd.DataFrame(docs_clean[i], columns = ['token'])
#     for category in constructs:
#         embedding_category = constructs_d.get(category)
#         embedding_category = np.array(embedding_category, dtype=float)
#         embeddings_tokens_doc_i = np.array(embeddings_tokens_doc_i, dtype=float)
#         if embeddings_tokens_doc_i.shape[0] == 0: #happens when there is an empty str
#             embeddings_tokens_doc_i = [np.zeros(embedding_category.shape[0])]
#         cosine_scores = cosine_similarity(embedding_category, embeddings_tokens_doc_i)
#         # each token is a row, and each col is a construct being measured for that token.             
#         df_scores_category_all[category] = np.array(cosine_scores, dtype = float)[0]#pd.DataFrame(cosine_scores, columns = ['category'])


#         # df_scores_category = pd.DataFrame([docs_clean[i], np.array(cosine_scores[0])]).T
#         # df_scores_category.columns = ['token', category]
#         # df_scores_category = df_scores_category.sort_values(by='token')
#         # # df_scores_category_all= df_scores_category_all.merge(df_scores_category, on='token', how = 'outer')
#         # df_scores_category_all.append(df_scores_category)
#     # df_scores_category_all = pd.concat(df_scores_category_all, axis=1)
#     df_scores_category_all = df_scores_category_all[constructs].astype(float)

#     # display(df_scores_category_all)
#     feature_vectors_mean.append(df_scores_category_all.mean())
#     feature_vectors_median.append(df_scores_category_all.median())
#     feature_vectors_max.append(df_scores_category_all.max())

# feature_vectors_mean = pd.concat(feature_vectors_mean,axis=1).T
# feature_vectors_mean.columns = [n+'_mean' for n in feature_vectors_mean.columns]

# feature_vectors_median = pd.concat(feature_vectors_median,axis=1).T
# feature_vectors_median.columns = [n+'_median' for n in feature_vectors_median.columns]

# feature_vectors_max = pd.concat(feature_vectors_max,axis=1).T
# feature_vectors_max.columns = [n+'_max' for n in feature_vectors_max.columns]

# feature_vectors = pd.concat([feature_vectors_mean, feature_vectors_median, feature_vectors_max],axis=1)
# df[feature_vectors.columns.tolist()] = feature_vectors.values
# # feature_cols = list(set(feature_vectors.columns)-set(['subreddit','author','date','docs','docs_clean']))
# # feature_cols.sort()
# # feature_vectors= feature_vectors[['subreddit','author','date','docs','docs_clean']+feature_cols]


In [ ]:
df = pd.read_csv(input_dir+f'train10_train_concurrent_metadata_100perconstruct_with_messages_preprocessed_23-03-20T17-50-34.csv')

In [ ]:
lexicon_embeddings = {}
print('encoding...')
embeddings_tokens_doc = vectorize(docs_final, list_of_lists=True, embedding_type = embedding_type, model_name = model_name) # 10 s for list of tokens for 5200 docs
# encode tokens of each doc
np.save(npy_filepath, embeddings_tokens_doc)


In [ ]:
lexicon.keys()

In [ ]:
constructs_suicide_risk_lexicon_weighted_centroid = pd.read_csv('./../data/lexicons/suicidal_thoughts_and_behaviors/weighted_centroids_22-12-04T01-06-02.csv', index_col = 0)

In [ ]:
%%time

run_this = True

lexicons_dir = './../data/lexicons/'
embeddings_path = lexicons_dir+'embeddings_tokens_all-MiniLM-L6-v2_22-12-02T17-32-05.json'
tokens_path = lexicons_dir+'suicidal_thoughts_and_behaviors/concurrent_validity_tokens_cosine_similarity_22-12-02T17-43-57.csv'



if run_this:
    for approach_embedding_name in approach_embedding_names:
        if 'glove' in approach_embedding_name:
            model_name = 'glove'

        elif 'minilm' in approach_embedding_name:
            model_name = 'all-MiniLM-L6-v2'
        elif 'psychbert' in approach_embedding_name:
            model_name = 'mnaylor/psychbert-cased'
        
        embedding_type = embedding_name_type.get(model_name)
        # if model_name in [
        #     # 'mnaylor/psychbert-cased',# cannot run on Mac M1, will run on colab: vectorize(docs_clean_joined, package = 'transformers', model_name = 'mnaylor/psychbert-cased', embedding_type = 'document')
        #     # 'all-MiniLM-L6-v2',
        #     # 'glove',
        #                 ]:
        #     continue # skip      
        print('approach_embedding_name: ', approach_embedding_name, 'model_name:', model_name, 'embedding_type: ', embedding_type)

        len_docs = len(docs_clean)
        # print(len_constructs , docs_per_construct )
        
        if '_w' in approach_embedding_name:
            docs_final = df['docs_clean_w_w'].values
            # df = pd.read_csv(output_dir+'feature_vectors_16constructs_7subreddits_156docs_w_w_minilm_22-10-09T02-19-07.csv', index_col = 0)
            # df = df.iloc[:, :5]
        elif '_c' in approach_embedding_name:
            docs_final = df['docs_clean_w_c'].values
            # df = pd.read_csv(output_dir+'feature_vectors_16constructs_7subreddits_156docs_w_c_minilm_22-10-09T02-19-07.csv', index_col = 0)
            # df = df.iloc[:, :5]
        df['docs_final']=docs_final


        type_of_document_tokenization = '_'.join(approach_embedding_name.split('_')[-2:]) #w, c

        
        npy_filepath = output_dir+f'army_starrs_{len(constructs)}constructs_{len_docs}docs_{type_of_document_tokenization}_embeddings.npy'
        try:
            embeddings_tokens_doc = np.load(npy_filepath,
                                            allow_pickle=True)
            print('loaded from prior run')

        except:
            print('did not find: ',npy_filepath)
            print('encoding...')
            embeddings_tokens_doc = vectorize(docs_final, list_of_lists=True, embedding_type = embedding_type, model_name = model_name) # 10 s for list of tokens for 5200 docs
            # encode tokens of each doc
            np.save(npy_filepath, embeddings_tokens_doc)

        filename = model_name.split('/')[-1]

        if approach_embedding_name.startswith('wl_'):
            # centroid weighted by cosine sim to construct label             

            constructs_d = {}
            # Load embedings for construct
            with open(embeddings_path, 'r') as json_file:
                lexicons_embeddings = json.load(json_file)

            lexicons_tokens = pd.read_csv(tokens_path, index_col = 0)
            for construct in lexicons_tokens['construct'].unique():
                weighted_centroid = constructs_suicide_risk_lexicon_weighted_centroid[construct].values
                constructs_d[construct]=weighted_centroid
                # lexicons_tokens_i = lexicons_tokens[lexicons_tokens['construct']==construct]
                # tokens_i = lexicons_tokens_i['token'].values
                # scores_i = lexicons_tokens_i['score'].values
                # embeddings_i = np.array([lexicons_embeddings.get(token) for token in tokens_i])
                # weighted_centroid = np.average(embeddings_i, axis=0, weights=scores_i)
                
                
            print('loaded dict of construct embeddings')

        else:
            try: 
#               # TODO:
                with open(output_dir+f'constructs{len(constructs)}_{approach_embedding_name}.pkl', 'rb') as f:
                    constructs_d = pickle.load(f)
                print('loaded dict of construct embeddings')
            except:
                print('encoding construct embeddings...')    
                # encode constructs     
                constructs_d = {}
                embeddings_constructs = vectorize(constructs, embedding_type = embedding_type, model_name = model_name)
                for category, embedding in zip(constructs, embeddings_constructs):
                    constructs_d[category] = embedding
                with open(output_dir+f'constructs{len(constructs)}_{approach_embedding_name}.pkl', 'wb') as f:
                    pickle.dump(constructs_d, f)

        # compute similarity (extract features)    
        # embeddings_tokens_doc = np.load(output_dir+f'army_starrs_5_{model_name.split('/')[-1]}_embeddings.npy')
        # constructs_d['hallucinating'] = constructs_d['hallucinating_hallucination']
        # del  constructs_d['hallucinating_hallucination']

        feature_vectors = df_similarity_token_category(embeddings_tokens_doc, constructs_d, df, docs_final, summary_stats = None)
        feature_vectors.to_csv(output_dir+f'feature_vectors_{len(constructs)}constructs_{len_docs}docs_{approach_embedding_name}_{ts}.csv')
